In [1]:
import os
from time import time

import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal

import multiprocessing as mp

import rioxarray as rxr
from numba import njit
from shapely.ops import unary_union

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Whisker
from bokeh.io import output_notebook, export_png
from bokeh.palettes import Colorblind3
from bokeh.layouts import gridplot
# from bokeh.layouts import gridplot, row, column
output_notebook()

import warnings
warnings.filterwarnings('ignore')

Loading BokehJS ...

In [2]:
from whitebox.whitebox_tools import WhiteboxTools

wbt = WhiteboxTools()
wbt.verbose = False
# wbt.verbose = True

## Evaluate the effect of dem resolution on basin representation and attribute estimation

As the basin scale decreases, methodological choices begin to have a significant impact on the number of cells captured and used to represent a basin. In addition, some attributes are affected by DEM resolution, in particular terrain attributes.  Here we investigate two examples. 

Consider a square grid intersected by an arbitrary curvilinear loop.  If we color the grid cells intersected by the line red, and we colour any cell inside the closed loop blue, let $P_{edge}$ the percentage of edge cells be the number of red cells divided by the number of red plus blue cells.

Now if we hold the loop constant and change the grid cell dimension $d_{grid}$, intuitively $P_{edge}$ will increase as the grid cell size increases since eventually the loop will be encompassed by a single grid cell.  $P_{edge}$ will approach 1 as $d_{grid}$ increases.

When we extract basin attributes from a geospatial raster using a basin polygon, at what point does a choice affecting the number of edge pixels become significant?  At some combination of raster resolution and basin size, the proportion of edge pixels will be significant.  The purpose of this exercise is to compare DEM datasets of two different resolutions to get a sense of the basin size at which the edge pixel method chosen becomes significant to the representativeness of the sample of raster pixels used to compute basin attributes.

## tl;dr

Compare 30m vs. 90m dem using a large sample of basin polygons over a wide scale to see when the basin is mostly edge pixels that are either included or excluded depending upon the clipping method.

Once the basin clipping is done, the mean slope is computed for each basin from two different DEM sources to compare the effect of DEM resolution on mean basin slope calculation.

## Method

The method is as follows:
1. Select a random sample of 10k basin polygons from the BCUB polygon set.
2. Use each polygon as a clipping mask and create a temporary clipped DEM to represent the basin pixels. Save these as temporary files because they will be ingested by Whiteboxtools "Slope" tool in the subsequent slope comparison.
3. For each clipped raster:    
   a. Find all non-Nan indices in the clipped raster.  An edge pixel is one which has at least one NaN neighbour.  
   b. Count all edge pixels and divide by the total number of non-NaN indices.  
   c. The process assumes there are no data gaps in the basin prior to clipping.  This is checked by first counting NaN values for pixels within the polygon and asserting equal to zero.

In [15]:
BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, 'processed_data')
# HYSETS_DIR = os.path.join(BASE_DIR, '/home/danbot2/code_5820/large_sample_hydrology/common_data/HYSETS_data/'

# This assumes you've downloaded a basin polygon set 
# and have processed the DEM for the same region 
# with both the 3DEP and EENV DEM
# here we'll do it for 08D (central coast)
region_code = 'FRA'
BASIN_DIR = os.path.join(DATA_DIR, 'derived_basins')
buffer = 0
# update with the path of processed DEM
# these scripts would be already run in the dataset replication
# see the following scripts in setup_scripts/
#     * get_3DEP_DEM.py   <--retrieves the USGS 3DEP dem
#     * get_EENV_DEM.py   <--retrieves the EarthEnv 90m dem
#     * clip_region_DEM.py <-- takes the assembled tiles and creates 

DEM_PATH = '/home/danbot2/code_5820/large_sample_hydrology/common_data/DEM_data/processed_dem/'


In [4]:
temp_folder = os.path.join(BASE_DIR, 'validation/tmp')
tmp_basins = os.path.join(temp_folder, 'basin_polygons/')

if not os.path.exists(tmp_basins):
    os.makedirs(tmp_basins)

In [5]:
def compute_slope_on_region_raster(region_code, DEM_source):
    """
    Compute the slope on the same region using two different resolution input DEMs.
    """
    if region_code == 'FRA':
        region_code = 'Fraser'
    src_dem_fname = f'{DEM_source}/{region_code}_EENV_DEM_3005_res1.tif'
    if 'USGS' in src:
        src_dem_fname = f'{DEM_source}/{region_code}_{DEM_source}_3005_res1.tif'
    # compute the mean slope of the clipped basin raster
    output_fname = src_dem_fname.replace('res1.tif', 'slope.tif')
    output_fpath = os.path.join(DEM_PATH, output_fname)
    region_fpath = os.path.join(DEM_PATH, src_dem_fname)
    if not os.path.exists(output_fpath):
        wbt.slope(
            region_fpath, 
            output_fpath, 
            zfactor=None, 
            units="degrees", 
            callback=None
        )
   
    return output_fpath

In [6]:
region_slope_paths = []
for src in ['EENV_DEM90', 'USGS_3DEP']:
    region_slope_path = compute_slope_on_region_raster(region_code, src)
    region_slope_paths.append(region_slope_path)

In [7]:
geom_fpath = f'{region_code}/{region_code}_basins_R0.parquet'
df = gpd.read_parquet(os.path.join(BASIN_DIR, geom_fpath))
df.reset_index(drop=True, inplace=True)
# the ID and id columns are vestiges of the batch processing for delineating polygons
df.drop(['ID', 'id'], inplace=True, axis=1)

In [8]:
# use the ppt_x and ppt_y columns as unique identifiers
df['ppt_idxs'] = list(zip(df['ppt_lon_m_3005'].astype(int), df['ppt_lat_m_3005'].astype(int)))
ppt_idxs_set = set(df['ppt_idxs'])

In [13]:
# select a random subset of 10K basin polygons
use_existing = True
sample_size  = 1e4

if not use_existing:
    # Create a unique identifier using 'ppt_x' and 'ppt_y' as a tuple
    sample_ids = np.random.choice(df['ppt_idxs'].values, size=int(sample_size), replace=False)
else:
    # Retrieve existing IDs from filenames in the temp folder
    existing_basin_files = os.listdir(os.path.join(temp_folder, 'basin_polygons'))
    existing_ids = set((int(e.split('_')[0]), int(e.split('_')[1])) for e in existing_basin_files)

    # Calculate how many more IDs are needed
    n_missing = max(0, sample_size - len(existing_ids))

    # Get all unique IDs from the DataFrame and find the remaining ones
    all_ids = set(map(tuple, df[['ppt_lon_m_3005', 'ppt_lat_m_3005']].values))
    remaining_ids = np.array(list(all_ids - existing_ids))

    # If more IDs are needed, select them randomly
    additional_idxs = np.random.choice(range(len(remaining_ids)), size=int(n_missing), replace=False) if n_missing > 0 else []
    additional_ids = remaining_ids[additional_idxs]

    # Combine existing and additional IDs
    sample_ids = np.array(list(existing_ids) + list(additional_ids))

    # Print the number of IDs and how many were added
    print(f'{len(sample_ids)} basin ids ({len(additional_ids)} added)')


10000 basin ids (0 added)


In [18]:
# get the selected rows of the df
# `sampled_ids` is an array of tuples: [(lat1, lon1), (lat2, lon2), ...]
# Convert `sampled_ids` into a DataFrame for easier merging
latlon_cols = ['ppt_lat_m_3005', 'ppt_lon_m_3005']
for ll in latlon_cols:
    df[ll] = df[ll].astype(int)
# Use boolean indexing to filter rows that match the lat/lon pairs
sample_ids = [tuple(id_pair) for id_pair in sample_ids]
selected_df = df[df['ppt_idxs'].isin(set(sample_ids))].copy()
# Use merge to select rows that match the lat/lon pairs in `sampled_ids`

## Clip the DEM with each polygon and create temporary raster files

In [17]:
def retrieve_raster(fpath):
    rds = rxr.open_rasterio(fpath, masked=True, mask_and_scale=True)
    crs = rds.rio.crs
    affine = rds.rio.transform(recalc=False)
    return rds, crs, affine

In [18]:
def clip_rasters(input_vars):
    ppt_x, ppt_y, polygon, DEM_source, buffer, region_code, temp_folder = input_vars

    dem_fname = f'{DEM_source}/{region_code}_USGS_3DEP_3005_res1.tif'
    dem_fpath = os.path.join(DEM_PATH, dem_fname)
    rc = region_code
    if region_code == 'FRA':
        rc = 'Fraser'
    src_dem_fname = f'{DEM_source}/{rc}_EENV_DEM_3005_res1.tif'
    if 'USGS' in src:
        src_dem_fname = f'{DEM_source}/{rc}_{DEM_source}_3005_res1.tif'
        
    input_raster_path = os.path.join(DEM_PATH, src_dem_fname)
    
    # If buffer is needed, calculate it using raster resolution
    if buffer != 0:
        dem, crs_obj, affine = retrieve_raster(input_raster_path)
        res = dem.rio.resolution()
        buff = np.sqrt(res[0]**2 + res[1]**2) if buffer == 1 else max(abs(res[0]), abs(res[1]))
        polygon = polygon.buffer(buff)

    # save the polygon to a temp file
    crs = 3005
    basin_fname = f'basin_polygons/{ppt_x}_{ppt_y}_b{buffer}_basin_{crs}.shp'
    basin_fpath = os.path.join(temp_folder, basin_fname)
    
    if not os.path.exists(basin_fpath):  
        basin_data = {
            'ppt_lon_m_3005': [ppt_x], 
            'ppt_lat_m_3005': [ppt_y], 
            'region_code': [region_code]
        }
        bdf = gpd.GeoDataFrame(basin_data, geometry=[polygon], crs=crs)   
        geom_type = bdf.geometry.values[0].geom_type

        if geom_type != 'Polygon':
            print(f'geom is {geom_type}')
            if geom_type == 'GeometryCollection':
                foo = gpd.GeoDataFrame(geometry=[polygon], crs=3005)
                bdf = foo.dissolve()                
                geom_type = bdf.geometry.values[0].geom_type
                bdf.to_file(basin_fpath)
            elif geom_type == 'MultiPolygon':
                print(f'multipolygon found: {basin_fpath}')
                bdf = gpd.GeoDataFrame(geometry=[polygon], crs=3005)
                bdf = bdf.explode(index_parts=False)
                bdf['area'] = bdf.geometry.area
                bdf = bdf[bdf['area'] >= 2E3]
                if len(bdf) == 0:
                    print(f'no polygon found {basin_fpath}')
                    raise Exception('no geoms left!!')      
                elif len(bdf) == 1:
                    bdf.to_file(basin_fpath)
                else:
                    raise Exception('too many geoms!!')  
                geom_type = bdf.geometry.values[0].geom_type
            if geom_type != 'Polygon':
                raise Exception('fix geom type!!')
        else:
            if not os.path.exists(basin_fpath):
                bdf.to_file(basin_fpath)
    # Clip the raster using the saved basin polygon
    raster_fname = f"{DEM_source}/{ppt_x}_{ppt_y}_clipped_{buffer}mbuff.tif"
    fpath_out = os.path.join(temp_folder, raster_fname)
    if not os.path.exists(fpath_out):
        g = gdal.Warp(fpath_out, input_raster_path, format="GTiff",
                      cutlineDSName=basin_fpath,
                      cropToCutline=True)
        g = None 

    return (fpath_out)

In [21]:
tstart = time()

# get all the ids
failed_ids = []
edge_dict = {}

for src in ['USGS_3DEP', 'EENV_DEM90']:
    tmp_raster_folder = os.path.join(temp_folder, src)
    if not os.path.exists(tmp_raster_folder):
        os.mkdir(tmp_raster_folder)
    t1 = time()
    print(f'{len(selected_df)} items selected for processing')
    inputs = [
        (x, y, geometry, src, buffer, region_code, temp_folder) 
        for x, y, (geometry,) in zip(selected_df['ppt_lon_m_3005'].values, selected_df['ppt_lat_m_3005'].values, selected_df[['basin_geometry']].itertuples(index=False, name=None))
    ]
    print(f'Processing {len(inputs)} with {src} DEM')

    # clip the dems to all basin polygons
    n_procs = 8
    p = mp.Pool(n_procs)
    print('Starting raster clipping:')
    clip_fpaths = p.map(clip_rasters, inputs)
    print(len(clip_fpaths))
    

10000 items selected for processing
Processing 10000 with USGS_3DEP DEM
Starting raster clipping:
10000
10000 items selected for processing
Processing 10000 with EENV_DEM90 DEM
Starting raster clipping:
10000


## Count Perimeter Cells

In [22]:
@njit
def edge_pixel_proportion(m):
    """
    The input m is a matrix representing the dem
    where the only nonzero values are indices that
    lie inside the basin.  
    Count the cells that have at least one NaN neighbour
    and compare to the total number of numeric cells.
    """
    
    # Get the number of rows and columns in the matrices
    # rows, cols = m1.shape
    (r1, c1) = m.shape
    # print(r1, c1)

    dem_px = 0
    edge_px = 0

    # Count non-zero elements in matrix
    for row in range(r1):
        for col in range(c1):
            if ~np.isnan(m[row, col]):
                dem_px += 1
                # check all neighbouring cells if any are nan
                indices = [(row - 1, col - 1), (row - 1, col), (row - 1, col + 1),
                           (row, col-1), (row, col + 1),
                           (row + 1, col-1), (row + 1, col), (row + 1, col + 1)]
                nan_nbr = False
                for r, c in indices:
                    if (r <= r1 - 1) & (r >= 0) & (c >= 0) & (c <= c1 - 1):
                        if np.isnan(m[r, c]):
                            edge_px += 1
                            break
    
    # Return the proportion of edge pixels
    if dem_px == 0:
        return 0
    return edge_px / dem_px

In [23]:
def process_basin_raster(f):
    x, y = f.split('/')[-1].split('_')[:2]
    raster, crs, affine = retrieve_raster(f)
    data = raster.data[0]
    pct_edge_px = edge_pixel_proportion(raster.data[0])
    return (x, y, pct_edge_px)

In [24]:
def get_or_create_results(src, base_dir, temp_folder):
    """
    Process rasters for a given source, saving intermediate results periodically.
    """
    tmp_raster_folder = os.path.join(temp_folder, src)
    result_file = os.path.join(temp_folder, f'{src}_intermediate_results.csv')
    processed_results = pd.DataFrame(columns=['x', 'y', f'pct_edge_cells_{src}'])

    # Load existing results if available
    if os.path.exists(result_file):
        processed_results = pd.read_csv(result_file)
        print(f"Resuming {src} with {len(processed_results)} existing results.")
    
    # Track already processed (x, y) pairs to avoid duplication
    processed_ids = set(zip(processed_results['x'], processed_results['y']))
    
    # Get all raster file paths and sort them
    raster_paths = sorted(
        os.path.join(tmp_raster_folder, r) for r in os.listdir(tmp_raster_folder)
    )
    # Filter out already processed rasters
    # Assuming filenames contain x and y coordinates in the format: '{x}_{y}_...'
    remaining_paths = [
        f for f in raster_paths 
        if (int(os.path.basename(f).split('_')[0]), int(os.path.basename(f).split('_')[1])) not in processed_ids
    ]
    total_to_process = len(raster_paths)

    # Process remaining files and save progress
    for i, f in enumerate(remaining_paths, 1):
        result = process_basin_raster(f)
        processed_results = pd.concat([processed_results, pd.DataFrame([result], columns=['x', 'y', f'pct_edge_cells_{src}'])])
        
        # Save every 100 results or at the end
        if i % 500 == 0 or i == len(remaining_paths):
            processed_results.drop_duplicates(subset=['x', 'y'], inplace=True)
            processed_results.to_csv(result_file, index=False)
            print(f"{src}: {len(processed_results)} out of {total_to_process} basins processed and saved.")

    return processed_results

In [25]:
# Skip processing if the final result already exists
edge_px_results = []
for src in ['EENV_DEM90', 'USGS_3DEP']:
    edge_pixel_result_fpath = os.path.join('data', f'{region_code}_perimeter_pixel_proportion_{src}.csv')
    base_dir = os.path.join(temp_folder, src)
    if os.path.exists(edge_pixel_result_fpath):
        result = pd.read_csv(edge_pixel_result_fpath)
        print(f" {len(result)} results for {src} already exist at {edge_pixel_result_fpath}")
    else:
        # Process rasters for the specified source
        t1 = time()
        result = get_or_create_results(src, base_dir, temp_folder)
        elapsed_time = time() - t1
        print(f"Processed {src} in {elapsed_time:.0f}s")
    
        # Save the processed results to a CSV file for the source
        result.to_csv(edge_pixel_result_fpath, index=False)
        print(f"Final results for {src} saved to {edge_pixel_result_fpath} (N={len(result)})")
    result.set_index(['x', 'y'], inplace=True)
    edge_px_results.append(result)    
    

Resuming EENV_DEM90 with 7965 existing results.
EENV_DEM90: 8465 out of 17964 basins processed and saved.
EENV_DEM90: 8965 out of 17964 basins processed and saved.
EENV_DEM90: 9465 out of 17964 basins processed and saved.
EENV_DEM90: 9965 out of 17964 basins processed and saved.
EENV_DEM90: 10001 out of 17964 basins processed and saved.
Processed EENV_DEM90 in 137s
Final results for EENV_DEM90 saved to data/FRA_perimeter_pixel_proportion_EENV_DEM90.csv (N=10001)
Resuming USGS_3DEP with 7964 existing results.
USGS_3DEP: 8464 out of 10000 basins processed and saved.
USGS_3DEP: 8964 out of 10000 basins processed and saved.
USGS_3DEP: 9464 out of 10000 basins processed and saved.
USGS_3DEP: 9964 out of 10000 basins processed and saved.
USGS_3DEP: 10000 out of 10000 basins processed and saved.
Processed USGS_3DEP in 924s
Final results for USGS_3DEP saved to data/FRA_perimeter_pixel_proportion_USGS_3DEP.csv (N=10000)


In [26]:
# Merge the two DataFrames on 'x' and 'y'
edge_px_df = pd.concat(edge_px_results, axis=1)
# Create a dictionary for fast lookups of drainage area based on (x, y) pairs from df
area_dict = dict(zip(zip(df['ppt_lon_m_3005'], df['ppt_lat_m_3005']), df['drainage_area_km2']))
# Map the drainage area using the (x, y) tuple as the key in edge_px_df
edge_px_df['area'] = edge_px_df.apply(lambda row: area_dict.get(row.name), axis=1)
print(len(edge_px_df))
edge_px_df.head()
edge_px_df.dropna(how='any', inplace=True)


10001


## Plot Results

In [27]:
def equiprobable_binning(data, param1, param2, samples_per_bin):
    # group deviation values by perimeter bin number
    df = data.copy()
    n_bins = int(len(df)/samples_per_bin)
    print(f'   Creating {n_bins} bins of {samples_per_bin} samples/bin (N={n_bins* samples_per_bin})')
    
    qc, edges = pd.qcut(df[param1], q=n_bins, precision=3, retbins=True)    
    edges1 = edges[1:]
    
    df['p_bin'] = np.digitize(df[param1], bins=edges1, right=True)
    
    bin_widths = [j-i for i, j in zip(edges[:-1], edges[1:])] 
    bin_centres = [(j+i)/2 for i, j in zip(edges[:-1], edges[1:])]
    evs = df[[param1, param2, 'p_bin']].groupby('p_bin').mean()

    evs = evs.reindex(range(1, len(edges)), fill_value=np.nan)

    evs['bin_width'] = bin_widths
    evs['bin_centre'] = (edges[1:] + edges[:-1]) / 2
    # evs['edges'] = edges
    evs['ubnd'] = df[[param1, param2, 'p_bin']].groupby('p_bin').quantile(0.95)[param2]
    evs['lbnd'] = df[[param1, param2, 'p_bin']].groupby('p_bin').quantile(0.05)[param2]
    evs['q1'] = df[[param1, param2, 'p_bin']].groupby('p_bin').quantile(0.25)[param2]
    evs['q3'] = df[[param1, param2, 'p_bin']].groupby('p_bin').quantile(0.75)[param2]
    evs['median'] = df[[param1, param2, 'p_bin']].groupby('p_bin').quantile(0.5)[param2]
    evs['mean'] = df[[param1, param2, 'p_bin']].groupby('p_bin').mean()[param2]
    evs.loc[0, 'ubnd'] = evs.loc[1, 'ubnd']
    evs.loc[0, 'lbnd'] = evs.loc[1, 'lbnd']     
    evs.dropna(how='any', inplace=True)
    return evs, edges

In [28]:
def binned_fig(fig, src, df, samples_per_bin=500):
    edge_pct = f'pct_edge_cells_{src}'
    evs, edges = equiprobable_binning(df, 'area', edge_pct, samples_per_bin)
    
    linetype = 'dashed'
    color = Colorblind3[0]
    color2 = Colorblind3[0]
    label = '90m (EarthEnv)'
    if src == 'USGS_3DEP':
        label = '30m (USGS 3DEP)'
        color = 'black'
        linetype = 'dotted'
        color2 = 'black'
    
    fig.toolbar.autohide = True
    
    classes = evs['bin_centre'].values
    ub = evs['ubnd'].values
    lb = evs['lbnd'].values
    source = ColumnDataSource(data=dict(base=classes, upper=ub, lower=lb))
    
    # outlier range
    w = Whisker(base='base', upper="upper", lower="lower", source=source,
                     line_color=color, line_alpha=0.8, line_width=1)
    w.upper_head.line_color = color
    w.lower_head.line_color = color

    if src == 'USGS_3DEP':
        fig.circle(evs['bin_centre'], evs['median'], 
                color=color2, size=6, fill_alpha=0,
                   legend_label=f'{label}')
    else:
        fig.square(evs['bin_centre'], evs['median'], 
                color=color2, size=6,
                   legend_label=f'{label}')

    fig.add_layout(w)
    return fig

In [30]:
fig = figure(width=800, height=650, title=f"", y_range=(0, 0.5),
           toolbar_location='above', x_axis_type='log')

for src in ['EENV_DEM90', 'USGS_3DEP']:
    fig = binned_fig(fig, src, edge_px_df, 600)
    
fig.xaxis.axis_label = 'Drainage Area [km²]'
fig.yaxis.axis_label = 'Fraction of cells at edge'

fig.xaxis.axis_label_text_font_size = '24pt'
fig.yaxis.axis_label_text_font_size = '24pt'
fig.xaxis.major_label_text_font_size = '22pt'
fig.yaxis.major_label_text_font_size = '22pt'
fig.legend.label_text_font_size = '22pt'
fig.yaxis.axis_label_text_font = "Bitstream Charter"
fig.xaxis.axis_label_text_font = "Bitstream Charter"
fig.xaxis.major_label_text_font = "Bitstream Charter"
fig.yaxis.major_label_text_font = "Bitstream Charter"
fig.legend.label_text_font = 'Bitstream Charter'

# fig.grid.visible = False
show(fig)

   Creating 13 bins of 600 samples/bin (N=7800)
   Creating 13 bins of 600 samples/bin (N=7800)


## Compute mean basin slope

At the beginning of this notebook we processed the slope for the full region.  Here we compute the mean slope for each basin using basin polygons as clipping masks and compare 30 and 90m dem sources to see the effect on a large sample of basins.

**NOTE**

In this example we use a random sample of 10K basins from the Frasier basin, extracted from the basin geometry file.  Note that the figure in the associated paper draws a random sample of 10K basins from the full study region.  In hindsight, it should have been seeded such that the figure could be replicated precisely.  It was not done, however at the bottom of this is a text output of the list of basin IDs so the figure below can at least be replicated. 

The distribution of basin slopes will vary based on the random sample drawn, but the point of the exercise is to show that the lower resolution DEM tends to compute lower slopes for the same basin, and this trend holds across all samples.  One limitation of the comparison is that we use the basin polygon derived from the higher resolution DEM, and we do not check to see if the lower resolution stream network identifies the same stream network.

In [9]:
df.columns

Index(['drainage_area_km2', 'ppt_lon_m_3005', 'ppt_lat_m_3005', 'ppt_acc',
       'Perimeter_km', 'Elevation_m', 'Aspect_deg', 'Slope_deg', 'region_code',
       'geometry', 'basin_geometry', 'centroid_geometry', 'ppt_idxs'],
      dtype='object')

In [10]:
type(df['ppt_idxs'][0])

tuple

In [27]:
def clip_slope_raster(input_data):
    x, y, buffer, crs, input_raster_fpath, DEM_source = input_data
    basin_fname = f'basin_polygons/{x}_{y}_b{buffer}_basin_{crs}.shp'
    basin_fpath = os.path.join(temp_folder, basin_fname)
    if not os.path.exists(basin_fpath):
        print('{basin_fpath.split("/")[-1]} not found.  Saving polygon.')
        basin = df[df['ppt_idxs'] == (x, y)].copy()
        basin.to_file(basin_fpath)
        print('    ...saved')

    # Clip the raster using the saved basin polygon
    raster_fname = f"{DEM_source}/{x}_{y}_clipped_slope_{buffer}mbuff.tif"
    fpath_out = os.path.join(temp_folder, raster_fname)
    if not os.path.exists(fpath_out):
        g = gdal.Warp(fpath_out, input_raster_fpath, format="GTiff",
                      cutlineDSName=basin_fpath,
                      cropToCutline=True)
        g = None 
    return fpath_out

In [30]:
# Iterate through the sources
all_slope_dfs = []
for src in ['EENV_DEM90', 'USGS_3DEP']:
    slope_fpath = os.path.join('data', f'slope_comparison_check_{src}.csv')

    # Load existing results or create a new DataFrame
    # Load or initialize the DataFrame
    if os.path.exists(slope_fpath):
        slope_df = pd.read_csv(slope_fpath)
        
        if 'ppt_idxs' in slope_df:
            slope_df[['x', 'y']] = pd.DataFrame(slope_df['ppt_idxs'].tolist(), index=slope_df.index)
        slope_df.set_index(['x', 'y'], inplace=True)
    else:
        slope_df = pd.DataFrame(columns=['x', 'y', src]).set_index(['x', 'y'])

    # Prepare inputs and filter out processed basins
    input_raster_fpath = [e for e in region_slope_paths if src in e][0]
    processed_indices = set(slope_df.index)
    slope_inputs = [
        (i[0], i[1], buffer, 3005, input_raster_fpath, src) 
        for i in sample_ids if (i[0], i[1]) not in processed_indices
    ]

    # Skip processing if all basins are already processed
    if not slope_inputs:
        print(f"All basins for source {src} already processed.")
        all_slope_dfs.append(slope_df)
        continue

    # Compute slope using multiprocessing
    n_procs = 12
    with mp.Pool(n_procs) as pool:
        slope_paths = pool.map(clip_slope_raster, slope_inputs)

    print('All rasters clipped')
    # Process and update the DataFrame, saving every 500 iterations
    
    for n, path in enumerate(slope_paths, 1):
        raster, _, _ = retrieve_raster(path)
        mean_slope = np.nanmean(raster.data[0])
         
        # Extract x and y from the filename (assuming these are in the first two parts)
        x, y = map(int, path.split('/')[-1].split('_')[:2])
        
        slope_df.loc[(x, y), src] = mean_slope        

        # Save progress every 500 iterations
        if n % 500 == 0:
            slope_df.dropna(inplace=True)
            slope_df.to_csv(slope_fpath)
            print(f'Processed {n}/{len(slope_paths)} x, y pairs for source {src}')

    # Final save
    slope_df.dropna(inplace=True)
    slope_df.to_csv(slope_fpath)
    # slope_df.set_index(['x', 'y'], inplace=True)
    all_slope_dfs.append(slope_df)
    print(f"Processing complete for source {src}. Results saved to {slope_fpath}.")

All rasters clipped
Processing complete for source EENV_DEM90. Results saved to data/slope_comparison_check_EENV_DEM90.csv.
All rasters clipped
Processed 500/10000 x, y pairs for source USGS_3DEP
Processed 1000/10000 x, y pairs for source USGS_3DEP
Processed 1500/10000 x, y pairs for source USGS_3DEP
Processed 2000/10000 x, y pairs for source USGS_3DEP
Processed 2500/10000 x, y pairs for source USGS_3DEP
Processed 3000/10000 x, y pairs for source USGS_3DEP
Processed 3500/10000 x, y pairs for source USGS_3DEP
Processed 4000/10000 x, y pairs for source USGS_3DEP
Processed 4500/10000 x, y pairs for source USGS_3DEP
Processed 5000/10000 x, y pairs for source USGS_3DEP
Processed 5500/10000 x, y pairs for source USGS_3DEP
Processed 6000/10000 x, y pairs for source USGS_3DEP
Processed 6500/10000 x, y pairs for source USGS_3DEP
Processed 7000/10000 x, y pairs for source USGS_3DEP
Processed 7500/10000 x, y pairs for source USGS_3DEP
Processed 8000/10000 x, y pairs for source USGS_3DEP
Processed

In [50]:
# Merge the two DataFrames on 'x' and 'y'
slope_comparison_df = pd.concat(all_slope_dfs, axis=1)
slope_comparison_df.dropna(inplace=True)
slope_comparison_df = slope_comparison_df.clip(lower=0, upper=45)
print(slope_comparison_df.min())
print(slope_comparison_df.max())
slope_comparison_df.head()


EENV_DEM90         0.0
USGS_3DEP     0.518217
dtype: object
EENV_DEM90    45.0
USGS_3DEP       45
dtype: object


,,EENV_DEM90,USGS_3DEP
x,y,,
1078226,785254,5.468461,6.970643
1232235,503642,27.010790,29.342936
1360939,526361,13.773086,14.443652
1148707,706058,18.970585,22.02626
970901,905578,9.420944,10.851622


In [57]:
ph1 = figure(title=f'', toolbar_location=None, 
             width=800, height=650)#, x_range=fig1.x_range)

hhist1, hedges1 = np.histogram(slope_comparison_df['EENV_DEM90'].values, bins=20, density=True)
hhist2, hedges2 = np.histogram(slope_comparison_df['USGS_3DEP'].values, bins=hedges1, density=True)

ph1.xgrid.grid_line_color = None
ph1.yaxis.major_label_orientation = np.pi/4
ph1.background_fill_color = "#fafafa"
ph1.yaxis.axis_label = 'P(X)'
ph1.xaxis.axis_label = 'Mean Slope [deg]'


ph1.quad(bottom=0, left=hedges1[:-1], right=hedges1[1:], top=hhist1, legend_label='90m (EarthEnv)', 
                  line_alpha=0.6, fill_alpha=0.5, color=Colorblind3[0], line_color="#3A5785")
# hh1 = ph1.quad(bottom=0, left=hedges1[:-1], right=hedges1[1:], top=hzeros1, alpha=0.5, **LINE_ARGS)

ph1.quad(bottom=0, left=hedges2[:-1], right=hedges2[1:], top=hhist2, legend_label='30m (USGS 3DEP)',
                  line_alpha=0.6, fill_alpha=0.5, color=Colorblind3[2], line_color="#3A5785")
# hh11 = ph1.quad(bottom=0, left=hedges2[:-1], right=hedges2[1:], top=hzeros2, alpha=0.5, **LINE_ARGS)
# hh2 = ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hzeros, alpha=0.1, **LINE_ARGS)
# hh2 = ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hzeros, alpha=0.1, **LINE_ARGS)

ph1.xaxis.axis_label_text_font_size = '24pt'
ph1.yaxis.axis_label_text_font_size = '24pt'
ph1.xaxis.major_label_text_font_size = '22pt'
ph1.yaxis.major_label_text_font_size = '22pt'
ph1.legend.label_text_font_size = '22pt'
ph1.yaxis.axis_label_text_font = "Bitstream Charter"
ph1.xaxis.axis_label_text_font = "Bitstream Charter"
ph1.xaxis.major_label_text_font = "Bitstream Charter"
ph1.yaxis.major_label_text_font = "Bitstream Charter"
ph1.legend.label_text_font = 'Bitstream Charter'

ph1.toolbar_location = 'right'
ph1.toolbar.autohide = True

In [58]:
# layout = column(ph1)
show(ph1)

## Evaluate the Shared Region Boundary Problem

The study region is broken into sub-regions, and the process of delineating all sub-basins in the BCUB begins with using the HydroBASINS polygons as a starting point.  The issue with these is they are derived from a different DEM source so the bounds are unique.  We want the regions to be delineated from a continuous source (3DEP 30m dem) and not let the HydroBASINS boundaries confine the basin delineation.

To address this, we implemented an iterative process doing the following:

1.  Running `merge_region_polygons.py` does a few things, but the relevant code to start is grouping the HydroBASINS polygons into sub-regions where no polygon has more than one runoff outflow, and no inflows at the edges.
2.  Buffer the region polygons and run `clip_region_dem.py` to clip the tiled vrt dem to each buffered region.
3.  Run the `process_flow_accumulation.py` script to derive the flow accumulation and stream networks for each region.
4.  Run the `adjust_region_polygons` function to derive a covering set of polygons for each region.  The output is a file named `<region_code>_adjusted_covering_basins.geojson`, where the `<region_code>` is the three letter code string for the region.
5.  Check each result from 4 against the clipped vector.  If the covering set hits the edge of the buffered clipping vector, you need to adjust the clipping bounds and repeat steps 2-4 until you have polygons that are defined by the algorithm and the DEM and not the clipping mask used at the start.  If anyone gets this far and is trying their hand at this process, I'm sorry.
6.  Where polygons overlap or have gaps, if both buffered clipping masks fully contain the gaps/overlaps, we leave them as is for uncertainty analysis -- delineating catchment boundaries from adjacent basins turns up anomalies.  Where the edge is defined clearly on one side but intersects with the clipping mask edge, we use the side that is fully defined to trim the edge that ran up against the clipping mask.



### Measuring shared boundary deviations

With a satisfactory set of covering polygons, we now want to make two comparisons:

1.  The number and distribution of deviations between the HydroBASINS (initial) shared boundaries and the updated boundaries, and
2.  The number and distribution of deviations in the updated set of region boundaries between shared bounds themselves.

In [ ]:
region_polygon_folder = os.path.join(os.path.dirname(os.getcwd()), 'input_data')
old_regions_fname = 'BCUB_regions_HydroBASINS.geojson'
new_regions_fname = 'BCUB_regions_merged_R0.geojson'

old_df = gpd.read_file(os.path.join(region_polygon_folder, old_regions_fname))
new_df = gpd.read_file(os.path.join(region_polygon_folder, new_regions_fname))

if not old_df.sindex:
    old_sindex = old_df.sindex
if not new_df.sindex:
    new_sindex = new_df.sindex

In [ ]:
old_df.head()

## 1.  Old bounds vs. New

I think we want to process each region individually.

For each region, find all touching / intersecting neighbours. Get the intersection and the gaps (symmetric difference isn't the correct approach here).



In [ ]:
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon

In [ ]:
# Function to remove holes from a polygon
def remove_holes(polygon):
    if polygon.interiors:
        return Polygon(polygon.exterior)
    else:
        return polygon

# Define a function to clean geometries
def make_valid(geom):
    if not geom.is_valid:
        geom = geom.buffer(0)
    return geom

def fill_holes(geometry):
    if geometry.geom_type == 'Polygon':
        return remove_holes(geometry)
    elif geometry.geom_type == 'MultiPolygon':
        return MultiPolygon([remove_holes(poly) for poly in geometry])
    else:
        return geometry

In [ ]:
def get_hydrobasins_intersections(target, nbr):
    """ for each target-neighbour pairing,
    get the intersection of the Hydrobasins region polygon
    with the target and the covering set of the neighbour,
    and then the intersection of the target covering set 
    with the Hydrobasins region polygon of the neighboring set. """
    hyd_target = old_df.loc[old_df['region_code'] == target].copy().explode(index_parts=False)
    hyd_nbr = old_df.loc[old_df['region_code'] == nbr].copy().explode(index_parts=False)

    nbr_df = new_df[new_df['region_code'] == nbr].copy().explode(index_parts=False)
    targ_df = new_df[new_df['region_code'] == target].copy().explode(index_parts=False)

    # 1. get the intersection of target (HydroBASINS) polygon with covering set of neighbour
    print(f'    ...{target} overlay on {nbr}.')
    int_df1 = gpd.overlay(hyd_target, nbr_df, how='intersection')
    # 2. get the intersection of neighbour (HydroBASINS) polygon with the covering set of the target
    print(f'    ...{nbr} overlay on {target}.')
    int_df2 = gpd.overlay(hyd_nbr, targ_df, how='intersection')

    # merge the two intersections
    print('    ...merging the intersections.')
    merged = gpd.GeoDataFrame(pd.concat([int_df1, int_df2]), crs=new_df.crs)
    print('    ....intersections merged.')
    return merged


def merge_neighbours(df1, df2):
    intersecting_gdf1 = gpd.sjoin(df1, df2, how='inner', op='intersects')
    union = intersecting_gdf1.unary_union
    union_gdf = gpd.GeoDataFrame(geometry=[union], crs=new_df.crs).explode(index_parts=False)
    union_gdf['area'] = union_gdf.geometry.area / 1e6
    union_gdf = union_gdf[union_gdf.index == union_gdf['area'].idxmax()]

    exterior = gpd.GeoDataFrame(geometry=[Polygon(e) for e in union_gdf.geometry.exterior], crs=new_df.crs)
    exterior['area'] = exterior.geometry.area
    exterior = exterior[exterior.index == exterior['area'].idxmax()]
    return exterior


def get_self_intersections(target, nbr_df):
    """ for each target-neighbour pairing,
    get the set of deviations from a perfectly shared boundary.
    This includes overlapping regions and gaps"""
    targ_df = new_df[new_df['region_code'] == target].copy().explode(index_parts=False)

    # Clean geometries
    targ_df['geometry'] = targ_df['geometry'].apply(make_valid)
    nbr_df['geometry'] = nbr_df['geometry'].apply(make_valid)    

    # 1. get the intersections between covering sets of target and neighbour polygons
    int_df = gpd.overlay(nbr_df, targ_df, how='intersection')
    
    # 2. get the holes in the shared boundary
    # first, we merge the two regions to get the outer boundary
    edges_1 = merge_neighbours(nbr_df, targ_df)
    edges_2 = merge_neighbours(targ_df, nbr_df)
    merged = gpd.GeoDataFrame(pd.concat([edges_1, edges_2]), crs=new_df.crs)
    merged = gpd.GeoDataFrame(geometry=[merged.unary_union], crs=new_df.crs).explode()
    
    exterior = gpd.GeoDataFrame(geometry=[Polygon(e) for e in merged.geometry.exterior if e is not None], crs=new_df.crs)
    exterior['area'] = exterior.geometry.area
    if exterior.empty:
        print('exterior is empty:')
        print(merged)
        return gpd.GeoDataFrame()
    exterior = exterior[exterior.index == exterior['area'].idxmax()]

    holes = gpd.overlay(exterior, merged, how='difference')
    # merge the two components?
    output = gpd.GeoDataFrame(pd.concat([int_df, holes]), crs=new_df.crs)
    return output

In [ ]:
for rc in sorted(list(set(old_df['region_code']))):
    if rc in ['HGW', 'VCI']:
        continue
    # get all the intersecting regions
    
    assert old_df.crs == new_df.crs
    target_region = old_df.loc[old_df['region_code'] == rc].copy().explode(index_parts=False)
    intersecting_regions = gpd.sjoin(old_df, target_region, how='inner', predicate='intersects')
    intersecting_regions = intersecting_regions[intersecting_regions['region_code_left'] != rc].copy()
    nbr_regions = intersecting_regions['region_code_left'].values

    print(f'processing {rc}: intersects with {", ".join(nbr_regions)}')

    basin_edge_output_fname = f'data/{rc}_HydroBASINS_bounds_comparison.geojson'
    if not os.path.exists(basin_edge_output_fname):
        merged_intersections = []
        for nbr in nbr_regions:
            merged = get_hydrobasins_intersections(rc, nbr)
            merged_intersections.append(merged)

        merged_df = gpd.GeoDataFrame(pd.concat(merged_intersections), crs=new_df.crs)
        merged_df.to_file(basin_edge_output_fname)

    # get intersections with neighbouring regions between updated sets
    boundary_dev_output_fname = f'data/{rc}_bounds_deviations_test.geojson'
    if not os.path.exists(boundary_dev_output_fname):
        nbr_sets = new_df[new_df['region_code'].isin(nbr_regions)].copy()
        boundary_deviations = get_self_intersections(rc, nbr_sets)
        boundary_deviations.to_file(boundary_dev_output_fname)


### Analyze the uncertainty in region bounds

Compare the distribution of 

1. deviations between the updated sub-region bounds and the initial HydroBASINS bounds, and
2. deviations between the updated sub-region bounds.

In [ ]:
def plot_area_distribution(df):
    """
    Creates a Bokeh plot illustrating the distribution of values in the 'area' column of the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing an 'area' column.

    Returns:
    None
    """
    area_values = np.sort(df['area'].dropna())
    # Determine equiprobable bins
    # num_bins = 20
    cdf = np.arange(1, len(area_values) + 1) / len(area_values)
    
    # bin_labels, bin_edges = pd.qcut(area_values, q=num_bins, retbins=True, labels=False, duplicates='drop')
    source = ColumnDataSource(data=dict(
        x=area_values,
        y=cdf
    ))

    return source
    

In [ ]:
plots = []
plot_no = 0
hb_dfs, bc_dfs = [], []
for rc in sorted(list(set(old_df['region_code']))):
    if rc in ['HGW', 'VCI']:
        print(f' No shared boundary for {rc}')
        continue
    
    print(f'Processing shared boundary analysis for {rc}')
    hb_fname = f'data/{rc}_HydroBASINS_bounds_comparison.geojson'
    bcub_fname = f'data/{rc}_bounds_deviations_test.geojson'
    hb_df = gpd.read_file(hb_fname)
    bc_df = gpd.read_file(bcub_fname)

    hb_df = hb_df.explode(index_parts=False)
    bc_df = bc_df.explode(index_parts=False)

    hb_df['area'] = hb_df.geometry.area / 1e6
    bc_df['area'] = bc_df.geometry.area / 1e6
    hb_df = hb_df[hb_df['area'] > 0.01]
    bc_df = bc_df[bc_df['area'] > 0.01]
    hb_dfs.append(hb_df)
    bc_dfs.append(bc_df)
    
    n_hb, n_bc = len(hb_df), len(bc_df)
    hb_df = hb_df[hb_df.geometry.area / 1e6 > 0.01]
    bc_df = bc_df[bc_df.geometry.area / 1e6 > 0.01]
    n_hb1, n_bc1 = len(hb_df), len(bc_df)
    print(f'    HydroBASINS: {n_hb1}/{n_hb} geometries after filtering area <= 0.01km2 (<1% of smallest polygon in BCUB dataset)')
    print(f'    BCUB: {n_bc1}/{n_bc} geometries after filtering area <= 0.01km2 (<1% of smallest polygon in BCUB dataset)')
    
    hb_source = plot_area_distribution(hb_df)
    bc_source = plot_area_distribution(bc_df)
    
    # Create a Bokeh figure
    p = figure(title=f"{rc} (N={len(hb_df)} HydroBASINS, {len(bc_df)} BCUB)",
               x_axis_label='Area [km²]', x_axis_type='log',
               y_axis_label='P(X)',
               height=400, width=600)
    
    # Add a quad glyph to the figure
    p.line(x='x', y='y', source=hb_source, line_width=2, line_color="#dc267f", legend_label="HydroBASINS")
    p.line(x='x', y='y', source=bc_source, line_width=2, line_color="#648fff", legend_label="BCUB")
    p.line(x=[1, 1], y=[0, 1], line_width=2, line_dash='dashed', line_color='black', legend_label='1 km²')
    p.legend.location = 'bottom_right'
    
    plots.append(p)

In [ ]:
layout = gridplot(plots, ncols=3, width=400, height=225)
show(layout)

Aggregate all the boundary deviations and plot the CDFs.  

Ensure duplicate geometries are dropped.  Show the total numbers of each class.

In [ ]:
hb_all_df = gpd.GeoDataFrame(pd.concat(hb_dfs), crs=new_df.crs)
bc_all_df = gpd.GeoDataFrame(pd.concat(bc_dfs), crs=new_df.crs)

In [ ]:
# drop duplicate geometries 
hb_all_df['geom_str'] = hb_all_df['geometry'].apply(lambda geom: geom.wkt)
bc_all_df['geom_str'] = bc_all_df['geometry'].apply(lambda geom: geom.wkt)
hb_all_df = hb_all_df.drop_duplicates(subset='geom_str')
bc_all_df = bc_all_df.drop_duplicates(subset='geom_str')

hb_all_df['area'] = hb_all_df.geometry.area / 1e6
bc_all_df['area'] = bc_all_df.geometry.area / 1e6

# drop all geometries smaller than 1% of the smallest basin in BCUB
hb_all_df = hb_all_df[hb_all_df['area'] > 0.01]
bc_all_df = bc_all_df[bc_all_df['area'] > 0.01]
hb_all_df.to_file('data/HydroBASINS_all_boundary_deviations.geojson')
bc_all_df.to_file('data/BCUB_all_boundary_deviations.geojson')
print(len(hb_all_df), len(bc_all_df))

In [ ]:
# plot the distribution of unique shared boundary deviations for the entire study region
hb_source = plot_area_distribution(hb_all_df)
bc_source = plot_area_distribution(bc_all_df)

# Create a Bokeh figure
p = figure(title='',#f"Deviations from shared boundaries  (> 0.01 km²)",
           x_axis_label=r'$$\text{Area} [\text{km}^2]$$', 
           x_axis_type='log', x_range=(10**-2.5, 10**3.35),
           y_axis_label=r'$$P(X)$$', 
           height=900, width=1200)

In [ ]:
def format_plot(p, hb_source, bc_source):
    # Set the title font size
    # p.title.text_font_size = '22pt'
    
    # Set the axis label font sizes
    p.xaxis.axis_label_text_font_size = '28pt'
    p.yaxis.axis_label_text_font_size = '28pt'
    
    # Set the tick label font sizes
    p.xaxis.major_label_text_font_size = '28pt'
    p.yaxis.major_label_text_font_size = '28pt'

    
    
    # Add lines to the figure with increased line width for better visibility
    p.line(x='x', y='y', source=hb_source, line_width=4, line_color="#dc267f", legend_label=f"HydroBASINS vs. BCUB (N={len(hb_all_df)})")
    latex_str = f"BCUB vs. BCUB (N={len(bc_all_df)})"
    p.line(x='x', y='y', source=bc_source, line_width=4, line_color="#648fff", legend_label=latex_str)
    p.line(x=[1, 1], y=[0, 1], line_width=3, line_dash='dashed', line_color='black', legend_label='1 km²')
    
    # Adjust legend font size and location
    p.legend.label_text_font_size = '24pt'
    p.yaxis.axis_label_text_font = "Bitstream Charter"
    p.xaxis.axis_label_text_font = "Bitstream Charter"
    p.xaxis.major_label_text_font = "Bitstream Charter"
    p.yaxis.major_label_text_font = "Bitstream Charter"
    p.legend.label_text_font = 'Bitstream Charter'
    p.legend.location = 'bottom_right'
    p.toolbar_location=None
    return p
    

In [ ]:
p = format_plot(p, hb_source, bc_source)
show(p)


In [ ]:
export_png(p, filename='edge_deviation_fig.png', width=1050, height=800)

In [ ]:
hb_median = np.percentile(hb_all_df.geometry.area/1e6, 50)
bc_median = np.percentile(bc_all_df.geometry.area/1e6, 50)
print(f'Median shared boundary deviation size:\n    HydroBASINS vs. BCUB = {hb_median:.4f} km² \n    BCUB vs. BCUB = {bc_median:.4f} km²')